In [ ]:
import pandas as pd
from datetime import datetime
df = pd.read_csv("../DATA/Tokyo house price.csv",encoding="cp932")

筛选出“住宅”或“共同住宅”的项目和宅地

In [5]:
# 筛选出“住宅”或“共同住宅”的项目
def contains_residential(x):
    if pd.isna(x):
        return False
    return '住宅' in x or '共同住宅' in x

# 使用apply方法来筛选数据
filtered_df = df[df['用途'].apply(contains_residential)]
filtered_df = filtered_df .query('種類 == "宅地(土地と建物)"')

选择特征

In [7]:
selected_columns = ['最寄駅：距離（分）', '取引価格（総額）', '面積（㎡）','建築年', '建物の構造','地区名',"建ぺい率（％）","容積率（％）",'取引時期']
columns_order = [ '取引時期','取引価格（総額）','最寄駅：距離（分）', '面積（㎡）', '建築年', '建物の構造', '地区名', '建ぺい率（％）', '容積率（％）']
filtered_df = filtered_df[columns_order]

提取建筑年份的年份部分并转换为整数
增加建筑年数项目

In [9]:
# 提取建筑年份的年份部分并转换为整数
filtered_df['建築年'] = filtered_df['建築年'].str.extract('(\d+)')
filtered_df = filtered_df.dropna()
filtered_df['建築年'] = filtered_df['建築年'].astype(int)
current_year = datetime.now().year
filtered_df['建築年数'] = current_year - filtered_df['建築年']

修改距离和面积的异常值

In [10]:
distance_mapping = {
    '30分～60分': 30,
    '1H～1H30': 60,
    '1H30～2H': 90,
    '2H～': 120,
    '2,000㎡以上': 120
}

# 将文本值映射成数字
filtered_df['最寄駅：距離（分）'] = filtered_df['最寄駅：距離（分）'].map(lambda x: distance_mapping[x] if x in distance_mapping else int(x))

# 定义映射关系
distance_mapping = {
    '2,000㎡以上': 2000
}
# 将文本值映射成数字
filtered_df['面積（㎡）'] = filtered_df['面積（㎡）'].map(lambda x: distance_mapping[x] if x in distance_mapping else int(x))


修改取引時期的格式

In [16]:
def convert_quarter(quarter_str):
    year = quarter_str[:4]
    quarter = quarter_str[-4]
    month = str(int(quarter) * 3)
    return f"{year}-{month}"

# 应用转换函数
filtered_df['转换后日期'] = filtered_df['取引時期'].apply(convert_quarter)
filtered_df.drop('取引時期',axis=1)
filtered_df['取引時期'] = filtered_df['转换后日期']
filtered_df.drop('转换后日期',axis=1)

,取引時期,取引価格（総額）,最寄駅：距離（分）,面積（㎡）,建築年,建物の構造,地区名,建ぺい率（％）,容積率（％）,建築年数
0,2023-6,340000000,4,100,2007,ＲＣ,飯田橋,80.0,500.0,17
4,2019-3,180000000,1,50,1976,ＲＣ,飯田橋,80.0,700.0,48
6,2018-3,250000000,3,80,1986,ＲＣ,飯田橋,80.0,500.0,38
10,2022-6,130000000,5,60,1960,ＲＣ,飯田橋,80.0,500.0,64
12,2020-6,100000000,3,40,1966,ＲＣ,飯田橋,80.0,700.0,58
...,...,...,...,...,...,...,...,...,...,...
302739,2008-0,32000000,13,130,2008,木造,大字平井,40.0,80.0,16
302741,2007-21,28000000,24,115,2007,木造,大字平井,60.0,200.0,17
302742,2007-21,31000000,16,130,2007,木造,大字平井,40.0,80.0,17
302744,2007-0,26000000,18,270,1980,木造,大字平井,40.0,80.0,44


In [21]:
filtered_df.to_csv('../exported_data.csv', index=False, encoding='utf-8')

In [20]:
filtered_df = filtered_df.drop('转换后日期',axis=1)
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 164090 entries, 0 to 302812
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   取引時期       164090 non-null  object 
 1   取引価格（総額）   164090 non-null  int64  
 2   最寄駅：距離（分）  164090 non-null  int64  
 3   面積（㎡）      164090 non-null  int64  
 4   建築年        164090 non-null  int64  
 5   建物の構造      164090 non-null  object 
 6   地区名        164090 non-null  object 
 7   建ぺい率（％）    164090 non-null  float64
 8   容積率（％）     164090 non-null  float64
 9   建築年数       164090 non-null  int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 17.8+ MB
